# Train ANN with DSM2 data


## Define hyper-parameters

In [ ]:
####################################################
############## Hyper-param definitions #############
####################################################

'''
Define the model architecture: (input layer will be build automatically)

Supported abbreviation-layer pairs (detailed definitions can be found in "Layer builders" section)
- 'lstm': keras.layers.LSTM
- 'res': resnet block (basic_1d)
- 'c1d': keras.layers.Conv1D
- 'td': keras.layers.TimeDistributed
- 'dr': keras.layers.Dropout
- 'f': keras.layers.Flatten
- 'g': keras.layers.GRU
- 'd': keras.layers.Dense 
- 'o': keras.layers.Dense

Usage of resnet blocks: res(num_of_filters)x(kernel_size)x(stride)x(stages)
stages: # of resnet units in the block
example: model_str_def = 'res10x3x1x1_f_d8_d2_o1'

Usage of 1D conv layers: c1d(num_of_filters)x(kernel_size)x(stride)
example: model_str_def = 'c1d10x3x1_c1d10x3x1_f_d8_d2_o1'

'''
# Note: numbers following 'd' or 'o' will be multiplied by [size of output] before being used as numbers of neurons in dense layer

''' 
4 architectures are available:
MLP; LSTM; GRU; ResNet; Res-LSTM; Res-GRU
'''
## 1. To train an MLP Network: (uncomment both 2 lines below):
# model_str_def = 'd8_d2_o1'
# input_prepro_option=1

# 2. To train an LSTM Network (uncomment both 2 lines below):
# model_str_def = 'lstm8_f_o1'
# input_prepro_option = 2


## 3. To train a GRU Network (uncomment both 2 lines below):
# model_str_def = 'g8_f_o1'
# input_prepro_option = 2

# ## 4. To train a ResNet (uncomment both 2 lines below):
# model_str_def = 'resnet_8_2'
# num_res_blocks=1
# resnet_mult = [int(x) for x in model_str_def.split('_')[1:]]
# input_prepro_option = 2

# # 5. To train a Res-LSTM (uncomment both lines below):
# model_str_def = 'res_lstm2_8_2'
# input_prepro_option = 2

# # 6. To train a Res-GRU (uncomment both lines below):
model_str_def = 'res_gru2_8_2'
input_prepro_option = 2


''' 
Grouping stations or not?
'''
# if group_stations == True, 29 stations will be split into 3 groups
# if group_stations == False, 29 stations will be considered as 1 group
group_stations = False
exclude_stations = ['rsac101','sss']

''' 
Fraction of training set
'''
train_frac = 0.7


'''
Define parameters for input pre-processing
- ndays: number of daily values in inputs
- window_size: length of averaging windows
- nwindows: number of moving averages
'''
if input_prepro_option ==1:
    # option 1: apply pre-defined average windowing:
    ndays=8
    window_size=11
    nwindows=10
elif input_prepro_option ==2:
    # option 2: directly use daily measurements as inputs
    ndays=118
    window_size=0
    nwindows=0
else:
    raise "input_prepro_option=%d is not supported, please select between {0, 1}" % input_prepro_option


# number of training epochs
# training will stop when reaching this number or test loss doesn't decrease for 50 epochs
epochs = 5000


# set to False if you just want to evaluate trained models
train_models = True
save_results = True
save_figs = False


####################################################
########## End of hyper-param definitions ##########
####################################################

# "dsm2_ann_inputs_20220204.xlsx": simulated dataset
# "dsm2_ann_inputs_20220215.xlsx": observed dataset at daily resolution
# "dsm2_ann_observed_15min.xlsx":  observed dataset at 15-min resolution

data_file = "dsm2_ann_inputs_20220215.xlsx"

# percentile thresholds for ranged results
percentiles = [0,0.75,0.95]  

initial_lr=0.001

## Install packages

In [ ]:
import os
import sys
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import pickle
import time
import re

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import layers
#import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score


In [ ]:
# !pip install openpyxl==3.0.5
# !pip install pandas==1.1.3

In [ ]:

# import holoviews as hv
# hv.extension('bokeh')

# !pip install -q hvplot
# !pip install -q panel

# import hvplot.pandas
# import panel as pn

## Read data

In [ ]:
google_drive_dir = 'Data_Models_Script_for_2nd_Manuscript'
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
gdrive_root_path = os.path.join('/content/drive','My Drive',google_drive_dir)
data_path = os.path.join(gdrive_root_path,data_file)
num_sheets = {"dsm2_ann_BaselineData_20220120.xlsx":8,
              "dsm2_ann_inputs_20220204.xlsx":9,
              "dsm2_ann_inputs_20220215.xlsx":9,
              "dsm2_ann_observed_1h.xlsx":9,
              "dsm2_ann_observed_15min.xlsx":9}
num_dataset = {"dsm2_ann_inputs_20220204.xlsx":'simulated',
              "dsm2_ann_inputs_20220215.xlsx":'observed',
              "dsm2_ann_observed_15min.xlsx":'observed_15min'}

sys.path.append(gdrive_root_path)

dflist = [pd.read_excel(data_path,i,index_col=0,parse_dates=True) for i in range(num_sheets[data_file])]

df_inpout = pd.concat(dflist[0:(num_sheets[data_file])],axis=1).dropna(axis=0)
dfinps = df_inpout.loc[:,~df_inpout.columns.isin(dflist[num_sheets[data_file]-1].columns)]
dfouts = df_inpout.loc[:,df_inpout.columns.isin(dflist[num_sheets[data_file]-1].columns)]

Mounted at /content/drive


## Filter stations

In [ ]:
# read station names
output_locations = list(dfouts.columns[~dfouts.columns.str.contains('_dup')])
for s in output_locations:
    if s.lower() in exclude_stations:
        output_locations.remove(s)
print('Selected %d stations:' % len(output_locations), output_locations)

Selected 23 stations: ['CHDMC006', 'CHSWP003', 'CHVCT000', 'OLD_MID', 'ROLD024', 'ROLD059', 'RSAC064', 'RSAC075', 'RSAC081', 'RSAC092', 'RSAN007', 'RSAN018', 'RSAN032', 'RSAN037', 'RSAN058', 'RSAN072', 'RSMKL008', 'SLCBN002', 'SLDUT007', 'SLMZU011', 'SLMZU025', 'SLSUS012', 'SLTRM004']


In [ ]:
dfinps.head()

,northern_flow,sjr_flow,exports,/HIST+GATE/RSAC128/POS/01JAN1953 - 01JAN2020/IR-YEAR/DWR-DMS-DSM2/,div+seep-drain_dcd+smcd,daily_max-min,sjr_vernalis_ec,sac_greens_ec
2000-01-01,14954.0,1581.0,10535.7,1.0,2069.44,5.1,829.6,167.08
2000-01-02,14928.0,1581.0,10489.3,1.0,2104.34,5.4,824.5,164.17
2000-01-03,14998.0,1581.0,10557.0,1.0,2092.58,5.7,819.4,169.58
2000-01-04,14746.0,1631.0,10639.4,1.0,2077.37,5.9,814.2,166.67
2000-01-05,14681.0,1674.0,10758.0,1.0,2035.92,6.1,809.1,170.00


In [ ]:
dfouts

,CHDMC006,CHSWP003,CHVCT000,OLD_MID,ROLD024,ROLD059,RSAC064,RSAC075,RSAC081,RSAC092,...,RSAN058,RSAN072,RSMKL008,SLCBN002,SLDUT007,SLMZU011,SLMZU025,SLSUS012,SLTRM004,SSS
2000-01-01,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.0
2000-01-02,530.885417,-2.000000,-2.000000,899.968750,-2.000000,-2.000000,15525.515625,7169.864130,-2.000000,410.656250,...,-2.000000,-2.000000,175.750000,-2.000000,-2.000000,9546.875000,-2.000000,-2.000000,-2.000000,-2.0
2000-01-03,560.114584,-2.000000,-2.000000,892.083333,-2.000000,-2.000000,15375.697917,6549.184783,3492.006250,374.010416,...,-2.000000,-2.000000,171.531250,-2.000000,-2.000000,9502.604167,-2.000000,-2.000000,-2.000000,-2.0
2000-01-04,501.718750,-2.000000,-2.000000,906.427083,-2.000000,-2.000000,15423.510417,6601.141304,3354.670168,469.680000,...,-2.000000,-2.000000,167.864583,-2.000000,-2.000000,9641.875000,-2.000000,-2.000000,-2.000000,-2.0
2000-01-05,491.854167,-2.000000,-2.000000,878.958333,-2.000000,-2.000000,15907.562500,7710.029771,3649.512816,430.270833,...,-2.000000,-2.000000,176.385417,-2.000000,-2.000000,9831.770833,-2.000000,-2.000000,-2.000000,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,455.062500,416.286458,332.854167,639.083333,427.833333,687.468750,10791.906250,5826.927083,2418.104167,310.302083,...,602.968750,634.364583,196.270833,8121.322917,610.854167,8447.697917,6019.458333,7720.145833,462.604167,-2.0
2019-12-28,472.354167,428.161458,335.770833,656.750000,443.270833,697.718750,10060.083333,5117.729167,1991.656250,307.802083,...,644.770833,648.114583,196.000000,7770.479167,606.697917,8405.416667,5926.854167,7621.979167,443.364583,-2.0
2019-12-29,483.083333,438.223958,342.291667,652.552083,439.927083,718.250000,9644.697917,4882.864583,1830.187500,308.885417,...,650.885417,664.062500,191.656250,7905.635417,601.197917,8397.437500,5802.781250,7538.031250,424.979167,-2.0
2019-12-30,480.906250,434.765625,347.927083,631.729167,446.635417,735.562500,9643.729167,4832.671264,1883.322917,272.500000,...,632.385417,661.187500,188.864583,6706.145833,598.166667,8325.843750,5637.583333,7326.812500,424.218750,-2.0


## Tensorboard Setup
A log directory to keep the training logs

Tensorboard starts a separate process and is best started from the command line. Open a command window and activate this environment (i.e. keras) and goto the current directory. Then type in
```
tensorboard --logdir=./tf_training_logs/ --port=6006
```

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./tf_training_logs/ --port=6006
root_logdir = os.path.join(os.curdir, "tf_training_logs")
tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)## Tensorflow Board Setup

## Model definitions

### ResNet Block Building Function

In [ ]:
parameters = {
    "kernel_initializer": "he_normal"
}

def basic_1d(
    filters,
    stage=0,
    block=0,
    kernel_size=3,
    numerical_name=False,
    stride=None,
    force_identity_shortcut=False
):
    """
    A one-dimensional basic block.
    :param filters: the output’s feature space
    :param stage: int representing the stage of this block (starting from 0)
    :param block: int representing this block (starting from 0)
    :param kernel_size: size of the kernel
    :param numerical_name: if true, uses numbers to represent blocks instead of chars (ResNet{101, 152, 200})
    :param stride: int representing the stride used in the shortcut and the first conv layer, default derives stride from block id
    """
    if stride is None:
        if block != 0 or stage == 0:
            stride = 1
        else:
            stride = 2


    if block > 0 and numerical_name:
        block_char = "b{}".format(block)
    else:
        block_char = chr(ord('a') + block)

    stage_char = str(stage + 2)

    def f(x):
        y = keras.layers.ZeroPadding1D(padding=1,name="padding{}{}_branch2a".format(stage_char, block_char))(x)
        y = keras.layers.Conv1D(filters,kernel_size,strides=stride,use_bias=False,
                                name="res{}{}_branch2a".format(stage_char, block_char),
                                **parameters)(y)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation("relu", name="res{}{}_branch2a_relu".format(stage_char, block_char))(y)

        y = keras.layers.ZeroPadding1D(padding=1,name="padding{}{}_branch2b".format(stage_char, block_char))(y)
        y = keras.layers.Conv1D(filters,kernel_size,use_bias=False,
                                name="res{}{}_branch2b".format(stage_char, block_char),
                                **parameters)(y)
        y = keras.layers.BatchNormalization()(y)

        if block != 0 or force_identity_shortcut:
            shortcut = x
        else:
            shortcut = keras.layers.Conv1D(filters,1,strides=stride,use_bias=False,
                                           name="res{}{}_branch1".format(stage_char, block_char),
                                           **parameters)(x)
            shortcut = keras.layers.BatchNormalization()(shortcut)

        y = keras.layers.Add(name="res{}{}".format(stage_char, block_char))([y, shortcut])
        
        y = keras.layers.Activation("relu",name="res{}{}_relu".format(stage_char, block_char))(y)

        return y

    return f


def bottleneck_1d(
    filters,
    stage=0,
    block=0,
    kernel_size=3,
    numerical_name=False,
    stride=None,
):
    """
    A one-dimensional bottleneck block.
    :param filters: the output’s feature space
    :param stage: int representing the stage of this block (starting from 0)
    :param block: int representing this block (starting from 0)
    :param kernel_size: size of the kernel
    :param numerical_name: if true, uses numbers to represent blocks instead of chars (ResNet{101, 152, 200})
    :param stride: int representing the stride used in the shortcut and the first conv layer, default derives stride from block id
    """
    if stride is None:
        stride = 1 if block != 0 or stage == 0 else 2

    # axis = -1 if keras.backend.image_data_format() == "channels_last" else 1


    if block > 0 and numerical_name:
        block_char = "b{}".format(block)
    else:
        block_char = chr(ord('a') + block)

    stage_char = str(stage + 2)

    def f(x):
        y = keras.layers.Conv1D(filters,1,strides=stride,use_bias=False,
                                name="res{}{}_branch2a".format(stage_char, block_char),
                                **parameters)(x)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation("relu",name="res{}{}_branch2a_relu".format(stage_char, block_char))(y)

        y = keras.layers.ZeroPadding1D(padding=1,name="padding{}{}_branch2b".format(stage_char, block_char))(y)
        y = keras.layers.Conv1D(filters,kernel_size,use_bias=False,
                                name="res{}{}_branch2b".format(stage_char, block_char),
                                **parameters)(y)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation("relu",name="res{}{}_branch2b_relu".format(stage_char, block_char))(y)

        y = keras.layers.Conv1D(filters * 4, 1, use_bias=False,
                                name="res{}{}_branch2c".format(stage_char, block_char),
                                **parameters)(y)
        y = keras.layers.BatchNormalization()(y)

        if block == 0:
            shortcut = keras.layers.Conv1D(filters * 4, 1, strides=stride, use_bias=False,
                                           name="res{}{}_branch1".format(stage_char, block_char),
                                           **parameters)(x)
            shortcut = keras.layers.BatchNormalization()(shortcut)
        else:
            shortcut = x

        y = keras.layers.Add(name="res{}{}".format(stage_char, block_char))([y, shortcut])
        y = keras.layers.Activation("relu",name="res{}{}_relu".format(stage_char, block_char))(y)

        return y

    return f

### Layer builders

In [ ]:
"""# Custom loss function"""
def mse_loss_masked(y_true, y_pred):
    squared_diff = tf.reduce_sum(tf.math.squared_difference(y_pred[y_true>0],y_true[y_true>0]))
    return squared_diff/(tf.reduce_sum(tf.cast(y_true>0, tf.float32))+0.01)

# Define Sequential model
NFEATURES = (num_sheets[data_file]-1) # * (ndays + nwindows) 
def build_layer_from_string_def(s='i120',width_multiplier=1,
                                block=0,
                                force_identity_shortcut=False,
                                return_sequences_rnn=True):
    if s[0:4] == 'lstm':
        return layers.LSTM(units = int(s[4:])*width_multiplier, return_sequences=return_sequences_rnn, activation='sigmoid')
    elif s[0:3] == 'res':
        fields = s[3:].split('x')
        return basic_1d(filters=int(fields[0]),
                        stage=int(fields[3]),
                        block=block,
                        kernel_size=int(fields[1]),
                        stride=int(fields[2]),
                        force_identity_shortcut=force_identity_shortcut)
    elif s[0:3] == 'c1d':
        fields = s[3:].split('x')
        return keras.layers.Conv1D(filters=int(fields[0]), kernel_size=int(fields[1]), strides=int(fields[2]),
                                   padding='causal', activation='linear')
    elif s[0:2] == 'td':
        return keras.layers.TimeDistributed(keras.layers.Dense(int(s[2:]), activation='elu'))
    elif s[0:2] == 'dr':
        return keras.layers.Dropout(float(s[2:]))
    # elif s[0] == 'i':
    #     return keras.layers.InputLayer(input_shape=[int(s[1:]), NFEATURES])
    elif s[0] == 'f':
        return keras.layers.Flatten()
    elif s[0] == 'g':
        return keras.layers.GRU(int(s[1:])*width_multiplier, return_sequences=True, activation='relu')
    elif s[0] == 'd':
        return keras.layers.Dense(int(s[1:])*width_multiplier, activation='elu')
    elif s[0] == 'o':
        return keras.layers.Dense(int(s[1:])*width_multiplier, activation='linear')
    else:
        raise Exception('Unknown layer def: %s' % s)

### Model builder

In [ ]:
def build_model_from_string_def(strdef='i120_f_d4_d2_d1',width_multiplier=1):
    layer_strings = strdef.split('_')
    inputs = keras.layers.Input(shape=[int(layer_strings[0][1:]) * NFEATURES])
    x = None
    prev_conv_output_num_of_channels = None
    return_sequences_rnn = None
    for block,f in enumerate(layer_strings[1:-1]):
        if x is None:
            if f.startswith(('lstm','g')):         
                # these layers require 2D inputs and permutation
                x = layers.Reshape((ndays+nwindows,NFEATURES))(inputs)
                prev_conv_output_num_of_channels = NFEATURES
                x = layers.Permute((2,1))(x)
                return_sequences_rnn = layer_strings[block+2].startswith(('lstm','g','res','c1d'))
            elif f.startswith(('res','c1d')):
                # these layers require 2D inputs
                x = layers.Reshape((ndays+nwindows,NFEATURES))(inputs)
                prev_conv_output_num_of_channels = NFEATURES
            else:
                x = inputs


        x = build_layer_from_string_def(f,width_multiplier,block,
                                        force_identity_shortcut=(f.startswith('res') and prev_conv_output_num_of_channels==int(f[3:].split('x')[0])),
                                        return_sequences_rnn=return_sequences_rnn)(x)
        if f.startswith('lstm'):         
            prev_conv_output_num_of_channels=int(f[4:])
        elif f.startswith('res') or f.startswith('c1d'):
            prev_conv_output_num_of_channels=int(f[3:].split('x')[0])


    outputs = keras.layers.Dense(int(layer_strings[-1][1:])*width_multiplier, activation='linear')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=initial_lr), loss="mse")
    return model

def build_resnet_model(nhidden1=8, nhidden2=2, output_shape=1, act_func='sigmoid',
                       filters=num_sheets[data_file]-1, kernel_size=3, stride=1,
                       num_res_blocks=1):
    inputs = layers.Input(shape=NFEATURES* (ndays + nwindows))
    x = layers.Reshape((ndays+nwindows,NFEATURES))(inputs)
    for ii in range(num_res_blocks - 1):
        # TODO: think about conv filter numbers and kernel sizes
        intermediate_features = layers.ZeroPadding1D(padding=1,name="padding%d_branch2a" %ii)(x)
        intermediate_features = layers.Conv1D(filters=NFEATURES,kernel_size=2,strides=1,use_bias=False,
                          name="res%d_branch2a" %ii)(intermediate_features)
        intermediate_features = layers.BatchNormalization()(intermediate_features)
        intermediate_features = layers.Activation("relu", name="res%d_branch2a_relu" % ii)(intermediate_features)

        intermediate_features = layers.Conv1D(filters=NFEATURES,kernel_size=2,strides=1,use_bias=False,
                          name="res%d_branch2b" %ii)(intermediate_features)
        intermediate_features = layers.BatchNormalization()(intermediate_features)
        intermediate_features = layers.Activation("relu", name="res%d_branch2b_relu" % ii)(intermediate_features)

        shortcut = x
        x = layers.Add(name="res%d_add" % ii)([intermediate_features, shortcut])

    y = layers.ZeroPadding1D(padding=1,name="padding%d_branch2a" % num_res_blocks)(x)
    y = layers.Conv1D(filters,kernel_size,strides=stride,use_bias=False,
                                name="res%d_branch2a" % num_res_blocks)(y)
    y = layers.BatchNormalization()(y)
    y = layers.Activation("relu", name="res%d_branch2a_relu" % num_res_blocks)(y)

    y = layers.ZeroPadding1D(padding=1,name="padding%d_branch2b" % num_res_blocks)(y)
    y = layers.Conv1D(filters,kernel_size,use_bias=False,
                            name="res%d_branch2b" % num_res_blocks)(y)
    y = layers.BatchNormalization()(y)
    y = layers.Flatten()(y)
    y = layers.Dense(nhidden1, activation=act_func)(y)

    shortcut = inputs
    shortcut = layers.Dense(nhidden1, activation=act_func)(shortcut)

    y = layers.Add(name="res%d_add" % num_res_blocks)([y, shortcut])
    
    y = layers.Activation("relu",name="res_relu")(y)


    y = layers.Dense(nhidden2, activation=act_func)(y)
    outputs= layers.Dense(output_shape, activation=keras.activations.linear,name='output')(y)

    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=initial_lr), loss=mse_loss_masked)
    return model


def build_residual_lstm_model(nhidden1=8, nhidden2=2, output_shape=1, act_func='sigmoid',rnn_layer_name='lstm1',conv_init=None):
    layer_type, rnn_side_branch_multiplier, _ = re.split('(\d+)',rnn_layer_name)
    rnn_layer = layers.LSTM if layer_type == 'lstm' else layers.GRU
    inputs = layers.Input(shape=NFEATURES*(ndays+nwindows))
    x = layers.Reshape((ndays+nwindows,num_sheets[data_file]-1))(inputs)
    x = layers.Permute((2,1))(x)

    y = layers.Conv1D(8+10,1, activation='relu',
                      kernel_initializer=conv_init,
                      kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0, l2=0),
                      trainable=False)(x)

    y = layers.Flatten()(y)
    y = layers.Dense(nhidden1, activation=act_func)(y)
    y = layers.Dense(nhidden2, activation=act_func)(y)
    y = layers.Dense(output_shape, activation=keras.activations.linear,name='mlp_output')(y)

    shortcut = x
    shortcut = layers.Dense(nhidden1, activation=act_func)(shortcut)
    shortcut = rnn_layer(units = output_shape*int(rnn_side_branch_multiplier), activation=act_func,return_sequences=True)(shortcut)
    shortcut = layers.Flatten()(shortcut)
    shortcut = layers.Dense(output_shape, activation=keras.activations.linear,name='lstm_output')(shortcut)

    outputs = layers.Add(name="res_add")([y, shortcut])
    # outputs = layers.Activation("relu",name="res_relu")(outputs)
    outputs = layers.LeakyReLU(alpha=0.3,name="res_relu")(outputs)
    
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=initial_lr), loss=mse_loss_masked)
    return model


In [ ]:
import annutils

# create folders in Google Drive to save results
result_folders = ['models','results','images']
for result_folder in result_folders:
    if not os.path.exists(os.path.join(gdrive_root_path, result_folder)):
        os.makedirs(os.path.join(gdrive_root_path, result_folder))

Define station groups

In [ ]:
station_without_groups = {'all':output_locations}
station_with_groups = {'G1':['SSS','RSAC101','RSMKL008'],
                  'G2':['Old_River_Hwy_4','Middle_River_Intake','CCWD_Rock','SLTRM004','RSAN032','RSAN037','SLDUT007','ROLD024','RSAN058','RSAN072','OLD_MID','ROLD059','CHDMC006','CHSWP003','CHVCT000'],
                  'G3':['SLCBN002','SLSUS012','SLMZU011','SLMZU025','RSAC064','RSAC075','RSAC081','RSAN007','RSAC092','RSAN018','Martinez']}
final_groups = {False: station_without_groups, 
                True: station_with_groups}

## Define evaluation metrics

In [ ]:
from collections import defaultdict
eval_metrics = ['MSE', 'Bias', 'R', 'RMSD', 'NSE']

# key_stations = ['RSAC064','CCWDRock','CVPIntake', 'CCFBIntake','Emmaton','JerseyPoint']
if (data_file in ["dsm2_ann_inputs_20220204.xlsx","dsm2_ann_BaselineData_20220120.xlsx"]):
    key_stations = ['RSAC064','CCWD_Rock','CHDMC006', 'CHSWP003','RSAC092','RSAN018']
else:
    key_stations = ['RSAC064','CHDMC006', 'CHSWP003','RSAC092','RSAN018','ROLD024']#'SSS', 'RSAC101'

key_station_mappings = {'CCWD Rock': 'RSL',
                        # 'JerseyPoint':'RSAN018',
                        # 'Emmaton':'RSAC092', 
                        # 'CCFBIntake':'CHSWP003',
                        # 'CVPIntake':'CHDMC006',
                        # 'SLMZU025':'SLMZU025',
                        # 'ROLD059':'ROLD059',
                        'Middle River Intake':'MUP',
                        'Old River Hwy 4': 'OH4',
                        'OLD MID':'OLD_MID'}


def evaluate_sequences(target, pred, metrics,print_details=False):
    assert len(target) == len(pred), 'Target and predicted sequence length must equal.'
    valid_entries = target>0
    sequence_length = np.sum(valid_entries)
    # print('Total samples: %d, valid samples: %d' % (len(target), np.sum(valid_entries)))
    if np.any(sequence_length == 0):
        return {k: 0 for k in metrics}
    target=target[valid_entries]
    pred = pred[valid_entries]
    SD_pred = np.sqrt( np.sum((pred-np.mean(pred)) ** 2) /(sequence_length-1))
    SD_target = np.sqrt( np.sum((target-np.mean(target)) ** 2) /(sequence_length-1))

    eval_results = defaultdict(float)
    
    for m in metrics:
        if m =='MSE':
            eval_results[m] = ((target - pred)**2).mean()
        elif m =='Bias':
            eval_results[m] = np.sum(pred - target)/np.sum(target) * 100
        elif m == 'R':
            eval_results[m] = np.sum(np.abs((pred-np.mean(pred)) * (target - np.mean(target)))) / (sequence_length * SD_pred * SD_target)
        elif m == 'RMSD':
            eval_results[m] = np.sqrt(np.sum( ( ( pred-np.mean(pred) ) * ( target - np.mean(target) ) ) ** 2 ) / sequence_length)
        elif m == 'NSE':
            eval_results[m] = 1 - np.sum( ( target - pred ) ** 2 ) / np.sum( (target - np.mean(target) ) ** 2 )
    if print_details:
        print('(sum(pred - mean(pred)) x (target - mean(target))) =  %.4f' % np.sum((pred-np.mean(pred)) * (target - np.mean(target))))
        print('MSE =  %.4f' % eval_results[m])
        print('sum(pred - target) = %.4f' % np.sum(pred - target))
        print('sum(target) = %.4f' % np.sum(target))
        print('target standard deviation = %.6f, prediction standard deviation =%.6f' %(SD_target,SD_pred))
    return eval_results

## Train Models

In [ ]:
full_model_str_def = 'i%d_'%(ndays + nwindows) +model_str_def
for group_name, stations in final_groups[group_stations].items():
    start = time.time()
    if not train_models:
        break
    # # prepare dataset
    selected_output_variables = []
    for station in stations:
        for output in output_locations:
            if station in output:
                selected_output_variables.append(output)

    model_path_prefix = "mtl_%s_%s_%s_%s" % (group_name, full_model_str_def, num_dataset[data_file],str(int(train_frac*100))+'_train')

    print('Training model %s for %d stations: ' % (full_model_str_def, len(selected_output_variables)),end='')

    print([station.replace('target/','').replace('target','') for station in selected_output_variables])
    

    # create tuple of calibration and validation sets and the xscaler and yscaler on the combined inputs
    (xallc, yallc), (xallv, yallv), xscaler, yscaler = \
        annutils.create_training_sets([dfinps],
                                      [dfouts[selected_output_variables]],
                                      train_frac=train_frac,
                                      ndays=ndays,window_size=window_size,nwindows=nwindows)
    if model_str_def.startswith('resnet'):
        model = build_resnet_model(nhidden1=resnet_mult[0]*yallc.shape[1], nhidden2=resnet_mult[1]*yallc.shape[1], output_shape=yallc.shape[1],
                                   num_res_blocks=num_res_blocks)
    elif model_str_def.startswith('res'):
        conv_init = tf.constant_initializer(annutils.conv_filter_generator(ndays=8,window_size=11,nwindows=10))
        num_neurons_multiplier=[int(x) for x in model_str_def.split('_')[2:]]
        model = build_residual_lstm_model(num_neurons_multiplier[0]*len(output_locations),
                                 num_neurons_multiplier[1]*len(output_locations),
                                 output_shape=yallc.shape[1],
                                 act_func='sigmoid',
                                 rnn_layer_name=model_str_def.split('_')[1],
                                 conv_init=conv_init)
    else:
        model = build_model_from_string_def(full_model_str_def,width_multiplier=yallc.shape[1])

    display(model.summary())
    history = model.fit(
        xallc,
        yallc,
        epochs=epochs,
        batch_size=128,
        validation_data=(xallv, yallv),
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=50, mode="min", restore_best_weights=True),
            tensorboard_cb
        ],
        verbose=0,
    )
    # pd.DataFrame(history.history).hvplot(logy=True) # if you want to view the graph for calibration/validation training
    if save_results:
        annutils.save_model(os.path.join(gdrive_root_path, 'models', model_path_prefix) , model, xscaler, yscaler)

    print('Training time: %.2f seconds' % (time.time()-start)) 
    full_results={}
    range_results=defaultdict(defaultdict)


    df_inpout = pd.concat(dflist[0:(num_sheets[data_file])],axis=1).dropna(axis=0)
    dfinps = df_inpout.loc[:,~df_inpout.columns.isin(dflist[num_sheets[data_file]-1].columns)]
    dfouts = df_inpout.loc[:,df_inpout.columns.isin(dflist[num_sheets[data_file]-1].columns)]
    for group_name, stations in final_groups[group_stations].items():
        # prepare dataset
        selected_output_variables = []
        for station in stations:
            for output in output_locations:
                if station in output:
                    selected_output_variables.append(output)
        print('Testing MTL ANN for %d stations: ' % len(selected_output_variables),end='')
        # print([station.replace('target/','').replace('target','') for station in selected_output_variables])

        model_path_prefix = "mtl_%s_%s_%s_%s" % (group_name, full_model_str_def, num_dataset[data_file],str(int(train_frac*100))+'_train')

        # create tuple of calibration and validation sets and the xscaler and yscaler on the combined inputs
        (xallc, yallc), (xallv, yallv), xscaler, yscaler = \
            annutils.create_training_sets([dfinps],
                                        [dfouts[selected_output_variables]],
                                        train_frac=train_frac,
                                        ndays=ndays,window_size=window_size,nwindows=nwindows)

        annmodel = annutils.load_model(os.path.join(gdrive_root_path,'models', model_path_prefix),custom_objects={"mse_loss_masked": mse_loss_masked})

        train_pred = annmodel.model.predict(xallc)
        test_pred = annmodel.model.predict(xallv)
        
        all_target = np.concatenate((yallc,yallv),axis=0)
        all_pred = np.concatenate((train_pred,test_pred),axis=0)

        for ii, location in enumerate(selected_output_variables):
            if any(exclude_s in location.lower() for exclude_s in exclude_stations):
                continue
            if any([k.lower() in location.lower() for k in key_stations]):
                print_details = True
            else:
                print_details = False
            print(location)
            ## training results
            # y = dfouts.loc[:,location].copy()
            # y[y<0] = float('nan')
            # train_dfyp = pd.concat([y, dfp.loc[:,location]],axis=1).dropna()

            # print('Training R^2 ', r2_score(train_dfyp_norm.iloc[:,0],train_dfyp_norm.iloc[:,1]))

            train_results = evaluate_sequences(yallc.iloc[:,ii],train_pred[:,ii], eval_metrics,
                                            print_details=print_details)
            train_results['R^2'] = r2_score(train_pred[:,ii], yallc.iloc[:,ii])
            full_results['%s_train' %location] = train_results


            # test_dfyp_norm = pd.concat([dfouts_norm.loc[:,location], dfp_test_norm.loc[:,location]],axis=1).dropna()
            # test_dfyp = pd.concat([y, dfp_test.loc[:,location]],axis=1).dropna()


            eval_results = evaluate_sequences(yallv.iloc[:,ii], test_pred[:,ii], eval_metrics,
                                                print_details=print_details)
            eval_results['R^2'] = r2_score(test_pred[:,ii], yallv.iloc[:,ii])

            full_results['%s_test' %location] = eval_results

            # test_dfyp.columns=['target','prediction']
            
            # test_dfyp_norm['percent_mark'] = (test_dfyp['target'] > test_dfyp['target'].quantile(0.25)).astype(int) * 25
            # test_dfyp_norm.loc[test_dfyp['target'] > test_dfyp['target'].quantile(0.75),'percent_mark']=75
            for (lower_quantile, upper_quantile) in zip(percentiles,percentiles[1:]+[1,]):
                if print_details:
                    print('#'*20, '%d%% - %d%%' % ((lower_quantile*100, upper_quantile*100)),'#'*20)
                lower_threshold = np.quantile(all_target[:,ii], lower_quantile)
                upper_threshold = np.quantile(all_target[:,ii], upper_quantile)
                eval_results = evaluate_sequences(all_target[(all_target[:,ii] > lower_threshold) & (all_target[:,ii] <= upper_threshold),ii],
                                                all_pred[(all_target[:,ii] > lower_threshold) & (all_target[:,ii] <= upper_threshold),ii],
                                                eval_metrics,
                                                print_details=print_details)
                range_results[location][lower_quantile*100] = eval_results
        # create a pickle file on Google Drive and write results 
        result_path_prefix = "mtl_%s_%s" % (full_model_str_def, num_dataset[data_file])
        results_path = os.path.join(gdrive_root_path,"results/%s.pkl" % (result_path_prefix))

        f = open(results_path,"wb")
        pickle.dump(full_results,f)
        f.close()
        print('Numerical results saved to %s' % results_path)
        all_nse = []
        for station in full_results.keys():
            all_nse.append(full_results[station]['NSE'])
        print('Average NSE = %.5f' % np.mean(all_nse))
        #     if location == 'CCWD_Rock':
        #         break
        # if location == 'CCWD_Rock':
        #     break

Training model i118_res_gru2_8_2 for 23 stations: ['CHDMC006', 'CHSWP003', 'CHVCT000', 'OLD_MID', 'ROLD024', 'ROLD059', 'RSAC064', 'RSAC075', 'RSAC081', 'RSAC092', 'RSAN007', 'RSAN018', 'RSAN032', 'RSAN037', 'RSAN058', 'RSAN072', 'RSMKL008', 'SLCBN002', 'SLDUT007', 'SLMZU011', 'SLMZU025', 'SLSUS012', 'SLTRM004']
Randomly selecting 7188 samples for training, 0 for test
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 944)]        0           []                               
                                                                                                  
 reshape_4 (Reshape)            (None, 118, 8)       0           ['input_5[0][0]']                
                                                                                                  
 permute_4 (Permut

None

Training time: 86.74 seconds
Testing MTL ANN for 23 stations: CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  109.5472
MSE =  0.8204
sum(pred - target) = 33.1749
sum(target) = 1976.0916
target standard deviation = 0.153687, prediction standard deviation =0.138913
(sum(pred - mean(pred)) x (target - mean(target))) =  27.8361
MSE =  0.8222
sum(pred - target) = 4.7696
sum(target) = 492.4096
target standard deviation = 0.156522, prediction standard deviation =0.139134
#################### 0% - 75% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  56.3143
MSE =  0.6744
sum(pred - target) = 109.0801
sum(target) = 1487.8427
target standard deviation = 0.107732, prediction standard deviation =0.114477
#################### 75% - 95% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  1.9486
MSE =  -0.9954
sum(pred - target) = -40.5738
sum(target) = 732.0172
target standard deviation = 0.044144, prediction standard deviation =0.06209

None

Training time: 143.99 seconds
Testing MTL ANN for 23 stations: CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  111.5760
MSE =  0.8188
sum(pred - target) = -19.0700
sum(target) = 1973.3047
target standard deviation = 0.154833, prediction standard deviation =0.140860
(sum(pred - mean(pred)) x (target - mean(target))) =  27.1188
MSE =  0.8109
sum(pred - target) = -2.7580
sum(target) = 495.1965
target standard deviation = 0.151934, prediction standard deviation =0.139834
#################### 0% - 75% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  56.1387
MSE =  0.6768
sum(pred - target) = 63.6168
sum(target) = 1487.8427
target standard deviation = 0.107732, prediction standard deviation =0.115291
#################### 75% - 95% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  2.0214
MSE =  -1.4253
sum(pred - target) = -58.2766
sum(target) = 732.0172
target standard deviation = 0.044144, prediction standard deviation =0.062

...

Training time: 199.31 seconds
Testing MTL ANN for 23 stations: CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  111.4370
MSE =  0.8190
sum(pred - target) = -47.9974
sum(target) = 1968.5109
target standard deviation = 0.154696, prediction standard deviation =0.140651
(sum(pred - mean(pred)) x (target - mean(target))) =  27.5231
MSE =  0.8226
sum(pred - target) = -12.9304
sum(target) = 499.9903
target standard deviation = 0.152464, prediction standard deviation =0.139859
#################### 0% - 75% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  56.9648
MSE =  0.7045
sum(pred - target) = 33.0487
sum(target) = 1487.8427
target standard deviation = 0.107732, prediction standard deviation =0.115715
#################### 75% - 95% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  1.9556
MSE =  -1.5521
sum(pred - target) = -61.0935
sum(target) = 732.0172
target standard deviation = 0.044144, prediction standard deviation =0.06

None

Training time: 286.52 seconds
Testing MTL ANN for 23 stations: CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  114.1714
MSE =  0.7800
sum(pred - target) = 162.6751
sum(target) = 1985.0652
target standard deviation = 0.155174, prediction standard deviation =0.144043
(sum(pred - mean(pred)) x (target - mean(target))) =  26.5485
MSE =  0.7487
sum(pred - target) = 46.6848
sum(target) = 483.4360
target standard deviation = 0.150332, prediction standard deviation =0.140061
#################### 0% - 75% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  57.4679
MSE =  0.5107
sum(pred - target) = 231.0327
sum(target) = 1487.8427
target standard deviation = 0.107732, prediction standard deviation =0.118110
#################### 75% - 95% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  2.0744
MSE =  -0.7014
sum(pred - target) = -4.3014
sum(target) = 732.0172
target standard deviation = 0.044144, prediction standard deviation =0.065

None

Training time: 343.55 seconds
Testing MTL ANN for 23 stations: CHDMC006
(sum(pred - mean(pred)) x (target - mean(target))) =  106.9385
MSE =  0.8098
sum(pred - target) = -39.2538
sum(target) = 1971.0323
target standard deviation = 0.152827, prediction standard deviation =0.137210
(sum(pred - mean(pred)) x (target - mean(target))) =  29.1321
MSE =  0.8115
sum(pred - target) = -13.1270
sum(target) = 497.4689
target standard deviation = 0.159817, prediction standard deviation =0.143311
#################### 0% - 75% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  55.1912
MSE =  0.6870
sum(pred - target) = 44.4468
sum(target) = 1487.8427
target standard deviation = 0.107732, prediction standard deviation =0.113533
#################### 75% - 95% ####################
(sum(pred - mean(pred)) x (target - mean(target))) =  2.1062
MSE =  -1.6314
sum(pred - target) = -63.8471
sum(target) = 732.0172
target standard deviation = 0.044144, prediction standard deviation =0.06

# Evaluation

## Generate Plots

### Exceedance probability plots and time series plots for key stations

#### Re-order stations

In [ ]:
order = {v:i for i,v in enumerate([item for sublist in final_groups[True].values() for item in sublist])}
# for k,v in order.items():
#     print(k,':', v)

stations = []
simplified_station_names = []
for s in list(range_results.keys()):
    if all(exclude_s not in s.lower() for exclude_s in exclude_stations):
        new_s = s.split('-')[0].replace('_',' ').replace('-',' ')
        if new_s in key_station_mappings:
            new_s = key_station_mappings[new_s]
        stations.append(s)
        simplified_station_names.append(new_s)
station_orders = {}
for location in stations:
    for k,v in order.items():
        if k.lower() in location.lower():
            station_orders[location] = v
            break

# for k,v in station_orders.items():
#     print(k,':', v)

sorted_stations = sorted(stations, key=lambda x: station_orders[x])
simplified_station_names = [x for x, _ in sorted(zip(simplified_station_names, stations), key=lambda pair: station_orders[pair[1]])]

print(sorted_stations)
print(simplified_station_names)

['RSMKL008', 'SLTRM004', 'RSAN032', 'RSAN037', 'SLDUT007', 'ROLD024', 'RSAN058', 'RSAN072', 'OLD_MID', 'ROLD059', 'CHDMC006', 'CHSWP003', 'CHVCT000', 'SLCBN002', 'SLSUS012', 'SLMZU011', 'SLMZU025', 'RSAC064', 'RSAC075', 'RSAC081', 'RSAN007', 'RSAC092', 'RSAN018']
['RSMKL008', 'SLTRM004', 'RSAN032', 'RSAN037', 'SLDUT007', 'ROLD024', 'RSAN058', 'RSAN072', 'OLD_MID', 'ROLD059', 'CHDMC006', 'CHSWP003', 'CHVCT000', 'SLCBN002', 'SLSUS012', 'SLMZU011', 'SLMZU025', 'RSAC064', 'RSAC075', 'RSAC081', 'RSAN007', 'RSAC092', 'RSAN018']


#### Time Series Plots

In [ ]:
if save_figs:
    from matplotlib import ticker
    import matplotlib.dates as mdates
    from matplotlib.lines import Line2D
    myfontsize=15
    ncols=2
    fig_combined_timeseries, ax_combined_timeseries = plt.subplots(nrows=len(key_stations)//ncols,
                                                                ncols=ncols,
                                                                figsize=(6*ncols,3*len(key_stations)//ncols))
    fig_combined_timeseries.tight_layout(h_pad=3.5, w_pad=2)
    ii = 0
    for group_name, stations in final_groups[group_stations].items():
        # prepare dataset
        selected_output_variables = []
        for station in stations:
            for output in output_locations:
                if station in output:
                    selected_output_variables.append(output)
        print('Testing MTL ANN for %d stations: ' % len(selected_output_variables),end='')

        print([station.replace('target/','').replace('target','') for station in selected_output_variables],end='\n')
        model_path_prefix = "mtl_%s_%s_%s_%s" % (group_name, 'i%d_'%(ndays + nwindows) +model_str_def, num_dataset[data_file],str(int(train_frac*100))+'_train')

        annmodel = annutils.load_model(os.path.join(gdrive_root_path,'models', model_path_prefix),custom_objects={"mse_loss_masked": mse_loss_masked})

        ## training results
        dfp=annutils.predict(annmodel.model, dfinps, annmodel.xscaler, annmodel.yscaler,columns=selected_output_variables,
                            ndays=ndays,window_size=window_size,nwindows=nwindows)
        selected_key_stations = []
        for location in selected_output_variables:
            if any([k.lower() in location.lower() for k in key_stations]):
                print(location)
                selected_key_stations.append(location)
        selected_key_stations = sorted(selected_key_stations, key=lambda x: station_orders[x])

        for location in selected_key_stations:
            if any([k.lower() in location.lower() for k in key_stations]):
                print(location)
                simplified_station_name = location.split('-')[0].replace('_',' ').replace('-',' ')
                simplified_station_name = key_station_mappings.get(simplified_station_name) or simplified_station_name
                y = dfouts.loc[:,location].copy()
                y[y<0] = float('nan')


                # Observation vs. Prediction Time Series Plots
                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].plot(y.iloc[(ndays+nwindows*window_size-1):],'-',color='C0',alpha=0.8)
                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].plot(dfp.loc[:,location],'-',color='C3',alpha=0.8)
                if ii == ncols:
                    custom_lines = [Line2D([0], [0], color='C0'),
                                    Line2D([0], [0], color='C3')]
                    
                    ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].legend(custom_lines,
                                                                                    ['Target','Model Predictions'],
                                                                                    bbox_to_anchor=(1.28, 2.76),
                                                                                    fontsize=myfontsize)
                

                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].set_title(simplified_station_name,fontsize=myfontsize)
                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].set_xlim([df_inpout.index[0], df_inpout.index[-1]])
                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].tick_params(axis='x', labelsize=myfontsize)
                ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].tick_params(axis='y', labelsize=myfontsize)
                t = ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].text(df_inpout.index[int(df_inpout.shape[0]*0.4)],
                            ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].get_ylim()[1]*0.58,
                            '               Train       Test\nR           %.3f       %.3f\nNSE       %.3f       %.3f\n%% Bias  %.3f%%   %.3f%%' % (full_results['%s_train' %location]['R'],
                                                                    full_results['%s_test' %location]['R'],
                                                                    full_results['%s_train' %location]['NSE'],
                                                                    full_results['%s_test' %location]['NSE'],
                                                                    full_results['%s_train' %location]['Bias'],
                                                                    full_results['%s_test' %location]['Bias']),
                            fontsize=myfontsize-2)
                t.set_bbox(dict(facecolor='white', alpha=0.5, edgecolor='white'))
                if ii >= len(key_stations) - ncols:
                    ax_combined_timeseries[ii//ncols,int(ii-(ii//ncols)*ncols)].set_xlabel('Time')
                ii += 1
        print('Done')
    if save_figs:
        fig_combined_timeseries.savefig(os.path.join(gdrive_root_path,"images/%s_combined_time_series.png"% (model_path_prefix)),bbox_inches='tight',dpi=300)


#### Exceedance Prob Plots

In [ ]:
def add_subplot_axes(ax,rect,facecolor='w'): # matplotlib 2.0+
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3]
    subax = fig.add_axes([x,y,width,height],facecolor=facecolor)  # matplotlib 2.0+
    x_labelsize = subax.get_xticklabels()[0].get_size()
    y_labelsize = subax.get_yticklabels()[0].get_size()
    x_labelsize *= rect[2]**0.5
    y_labelsize *= rect[3]**0.5
    subax.xaxis.set_tick_params(labelsize=x_labelsize)
    subax.yaxis.set_tick_params(labelsize=y_labelsize)
    return subax

In [ ]:
if save_figs:
    from matplotlib import ticker
    import matplotlib.dates as mdates
    myfontsize=15
    ncols=2
    fig_combined_exceedance, ax_combined_exceedance = plt.subplots(nrows=len(key_stations)//ncols,
                                                                ncols=ncols,
                                                                figsize=(6*ncols,4*len(key_stations)//ncols))
    fig_combined_exceedance.tight_layout(h_pad=8, w_pad=2)
    ii = 0

    for group_name, stations in final_groups[group_stations].items():
        # prepare dataset
        selected_output_variables = []
        for station in stations:
            for output in output_locations:
                if station in output:
                    selected_output_variables.append(output)
        print('Testing MTL ANN for %d stations: ' % len(selected_output_variables),end='')

        print([station.replace('target/','').replace('target','') for station in selected_output_variables],end='\n')
        model_path_prefix = "mtl_%s_%s_%s_%s" % (group_name, 'i%d_'%(ndays + nwindows) +model_str_def, num_dataset[data_file],str(int(train_frac*100))+'_train')

        annmodel = annutils.load_model(os.path.join(gdrive_root_path,'models', model_path_prefix),custom_objects={"mse_loss_masked": mse_loss_masked})

        ## training results
        dfp=annutils.predict(annmodel.model, dfinps, annmodel.xscaler, annmodel.yscaler,columns=selected_output_variables,
                            ndays=ndays,window_size=window_size,nwindows=nwindows)
        
        selected_key_stations = []
        for location in selected_output_variables:
            if any([k.lower() in location.lower() for k in key_stations]):
                print(location)
                selected_key_stations.append(location)
        selected_key_stations = sorted(selected_key_stations, key=lambda x: station_orders[x])

        for location in selected_key_stations:
            simplified_station_name = location.split('-')[0].replace('_',' ').replace('-',' ')
            simplified_station_name = key_station_mappings.get(simplified_station_name) or simplified_station_name
            y = dfouts.loc[:,location].copy()
            y[y<0] = float('nan')

            # Combined exceedance probability plots
            fig_combined_exceedance, ax_combined_exceedance
            test_y_sorted = np.sort(y.iloc[(ndays+nwindows*window_size-1):].dropna().to_numpy())
            test_pred_sorted = np.sort(dfp.loc[:,location].clip(0,dfouts.loc[:,location].max()).to_numpy())

            # calculate the proportional values of samples
            p = 1. * np.arange(len(test_y_sorted)-1,-1,-1) / (len(test_y_sorted) - 1)
            p_pred = 1. * np.arange(len(test_pred_sorted)-1,-1,-1) / (len(test_pred_sorted) - 1)

            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].tick_params(axis='x', labelsize=myfontsize)
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].tick_params(axis='y', labelsize=myfontsize)

            # plot the sorted data:
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].plot(p, test_y_sorted)
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].plot(p_pred, test_pred_sorted,'--',color='C3')
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].ticklabel_format(axis='y',style='sci',scilimits=(0,0),useMathText=True)

            # f = ticker.ScalarFormatter(useOffset=False, useMathText=True)
            # g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.10e' % x))
            # ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].yaxis.set_major_formatter(ticker.FuncFormatter(g))


            if int(ii-(ii//ncols)*ncols) == 0:
                ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].set_ylabel('EC ' + r"$(\mu S/cm)$",fontsize=myfontsize)
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].yaxis.get_offset_text().set_fontsize(myfontsize)
            ylims = ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].get_ylim()
            if ii >= len(key_stations) - ncols:
                ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].set_xlabel('Exceedance Probability ' + r"$(\%)$",fontsize=myfontsize)
                text_yloc = ylims[0]-(ylims[1]-ylims[0])*0.37
            else:
                text_yloc = ylims[0]-(ylims[1]-ylims[0])*0.2

            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].text(0.48,
                                                                            text_yloc,
                                                                            '(%s)'%chr(97+ii),weight='bold',fontsize=myfontsize)
            if ii == ncols:
                plt.legend(['Target','Model Predictions'],fontsize=myfontsize,bbox_to_anchor=(-0.3, 2))
            ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)].set_title(simplified_station_name,fontsize=myfontsize)
            
            subax1 = add_subplot_axes(ax_combined_exceedance[ii//ncols,int(ii-(ii//ncols)*ncols)],[0.4,0.5,0.58,0.44]) # xloc, yloc, xwidth, ywidth
            # y.index = y.index.to_period('Y')
            # dfp.index = dfp.index.to_period('Y')
            subax1.plot(y.iloc[(ndays+nwindows*window_size-1):])
            subax1.plot(dfp.loc[:,location].clip(0,dfouts.loc[:,location].max()),'--',color='C3',alpha=0.8)
            # subax1.set_xlim(pd.Timestamp('1986-12-31'), pd.Timestamp('2023-01-01'))
            subax1.set_xticks(pd.date_range(start=dfp.index[0],
                                            end=dfp.index[-1],
                                            periods=5))
            subax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            plt.xticks(fontsize=myfontsize-3)

            # subax1.set_ylim(y.iloc[(ndays+nwindows*window_size-1):].min()*0.8,
            #                 y.iloc[(ndays+nwindows*window_size-1):].max()*1.2)
            min_x = int(y.iloc[(ndays+nwindows*window_size-1):].min()//100*100)
            max_x = int((y.iloc[(ndays+nwindows*window_size-1):].max()//100+1)*100)
            subax1.set_yticks(np.linspace(min_x,max_x,(max_x-min_x)//((max_x-min_x) // 3//100*100)))
            subax1.get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

            plt.yticks(fontsize=myfontsize-3)

            subax1.yaxis.get_offset_text().set_fontsize(myfontsize-3)

            # subax1.yaxis.set_major_locator(ticker.MaxNLocator(4))
            # subax1.xaxis.set_major_locator(ticker.MaxNLocator(6))
            ii += 1

        print('Done')
        fig_combined_exceedance.savefig(os.path.join(gdrive_root_path,"images/%s_combined_exceedance_prob.png"% (model_path_prefix)),bbox_inches='tight',dpi=300)


### Ranged result plots of R, NSE and Bias for key stations

In [ ]:
if save_figs:
    # plot percentile results for key stations
    myfontsize=14
    # train and test results plot in different figures


    # legends = ['{0:>3} ~ {1:<3}%'.format(s1, s2) for _ in datasets_to_plot for (s1, s2) in zip(percentiles[1:],percentiles[:-1])]

    # fig_name_prefix = os.path.join(result_path,'20220225_models/images/MTL_Range_Performance')
    plot_metrics=['r^2','Bias','RSR','NSE']
    fig, ax = plt.subplots(nrows=1,ncols=len(plot_metrics),figsize=(15,8))
    plt.subplots_adjust(wspace=0.3)

    for ii, metric in enumerate(plot_metrics):
        cmap='viridis'
        plot_metric = metric
        if metric=='r^2':
            plot_metric = 'R'
        elif metric=='RSR':
            plot_metric = 'NSE'
            cmap='viridis_r'
        elif metric =='Bias':
            # cmap = Pbias_map
            cmap = 'RdYlBu_r'
        to_plot=np.zeros([len(sorted_stations),len(percentiles)])
        for x_loc, station in enumerate(sorted_stations):
            for y_loc, percentile_range in enumerate(percentiles):
                if metric=='r^2':
                    to_plot[x_loc,y_loc] = range_results[station][int(percentile_range*100)][plot_metric]**2
                elif metric=='RSR':
                    to_plot[x_loc,y_loc] = np.sqrt(1-range_results[station][int(percentile_range*100)][plot_metric])
                else:
                    to_plot[x_loc,y_loc] = range_results[station][int(percentile_range*100)][plot_metric]
                if 'rsac064' in station.lower() and metric == 'Bias':
                    print(range_results[station][int(percentile_range*100)][plot_metric])
        # if metric=='r^2':
        #     current_plot = ax[ii].pcolor(to_plot,vmin=0, vmax=1)
        # else:
        current_plot = ax[ii].pcolor(to_plot,cmap=cmap)
        current_plot.set_clim(current_plot.get_clim()[0],max(current_plot.get_clim()[1],1))
        cbar = plt.colorbar(current_plot,ax=ax[ii],aspect=30,fraction=0.1, pad=0.04)
        cbar.ax.tick_params(labelsize=myfontsize-1) 

        if metric =='Bias':
            ax[ii].set_title('Percent Bias (%)',fontsize=myfontsize+2)
        else:
            ax[ii].set_title(r'${}$'.format(metric),fontsize=myfontsize+2)
        if ii == 0:
            ax[ii].set_ylabel('Study Location',fontsize=myfontsize+2)
            ax[ii].set_yticks(np.arange(len(sorted_stations))+0.5)
            ax[ii].set_yticklabels(simplified_station_names,fontsize=myfontsize-3)
            # ax.tick_params(axis='y', labelrotation = 45, size=0)        
        else:
            ax[ii].set_yticks([])

        ax[ii].set_xlabel('Range',fontsize=myfontsize+2)
        ax[ii].set_xticks(np.arange(len(percentiles))+0.5)
        ax[ii].xaxis.set_tick_params(size=0)
        ax[ii].set_xticklabels(['0-%d%%'%(percentiles[1]*100),'%d-%d%%'%(percentiles[1]*100,percentiles[2]*100),'%d-100%%'%(percentiles[2]*100)],
                            fontsize=myfontsize-3)

        ax[ii].text(1.4, ax[ii].get_ylim()[0]-3.5,
                    '(%s)'%chr(97+ii),weight='bold',fontsize=myfontsize)
    plt.savefig(os.path.join(gdrive_root_path,"images/%s_Range_Performance.png"% (model_path_prefix)),bbox_inches='tight',dpi=300)